In [291]:
import nltk
from nltk.corpus import brown
from collections import defaultdict
from collections import Counter
import numpy as np
import math

In [292]:
corp = brown.sents(categories='news')
brown_news_tagged = brown.tagged_sents(categories='news', tagset='universal')

In [293]:
flat_corp = [item for l in corp for item in l]

In [294]:
flat_brown = [item for l in brown_news_tagged for item in l]

In [295]:
b = Counter(flat_brown)

In [296]:
# count parts of speech for each word
b_ik = defaultdict(dict)
for w_pos, c in b.items():
    b_ik[w_pos[0]][w_pos[1]] = c

In [297]:
# count # of occurences of each POS
denom = defaultdict(int)
for w_pos, c in b.items():
    denom[w_pos[1]] += c

In [298]:
# count the bigrams
a_ij = defaultdict(dict)
for c in range(0, len(flat_brown) - 1):
    if flat_brown[c+1][1] in a_ij[flat_brown[c][1]]:
        a_ij[flat_brown[c][1]][flat_brown[c+1][1]] += 1
    else:
        a_ij[flat_brown[c][1]][flat_brown[c+1][1]] = 1

In [299]:
# word to number dictionary
o_index = dict(zip(b_ik, range(len(b_ik))))

In [300]:
# number to word dictionary
o_index_n = dict(zip(range(len(b_ik)), b_ik))

In [301]:
# POS to number dictionary
s_index = dict(zip(a_ij, range(len(a_ij))))

In [302]:
# number to POS dictionary
s_index_n = dict(zip(range(len(a_ij)), a_ij))

In [303]:
em = np.zeros((len(s_index), len(o_index)))
tm = np.zeros((len(s_index), len(s_index)))

In [311]:
lam = .1

In [312]:
# Construct emission matrix with Laplace Smoothing
for col in range(0, len(em[0])):
    for row in range(0, len(em)):
        pos_list = b_ik[o_index_n[col]]
        if (s_index_n[row] in pos_list):
            em[row][col] = (b_ik[o_index_n[col]][s_index_n[row]] + lam) * 1.0 / (denom[s_index_n[row]] + len(em) * lam)
        else:
            em[row][col] = (lam) / (denom[s_index_n[row]] + len(em) * lam)

In [313]:
# Constuct transition matrix with Laplace Smoothing
for row in range(0, len(tm)):
    for col in range(0, len(tm[0])):
        pos_list = a_ij[s_index_n[row]]
        if (s_index_n[col] in pos_list):
            tm[row][col] = (a_ij[s_index_n[row]][s_index_n[col]] + lam) * 1.0 / (denom[s_index_n[row]] + len(tm) * lam)
        else:
            tm[row][col] = (lam) / (denom[s_index_n[row]] + len(tm) * lam)

In [314]:
# gets argmax{S}
def argmax(col, tm):
    mx = (-1, -float("inf"))

    for row in range(len(col)):
        value = col[row]
        value += math.log(tm[row])

        if value > mx[1]:
            mx = (row, value)

    return mx

In [315]:
def viterbi(em, tm, obs, init):
    n = len(em); T = len(obs)
    # step 1: init n x T matrix
    ls = np.zeros((n, T))
    phi = np.zeros((n, T))


    # col 1 in ls -> log(pi_i*bi_o1)
    for r in range(len(init)):
        value = init[r]*em[r][o_index[obs[0]]]
        if value != 0:
            ls[r][0] = math.log(value)

    #step 2: fill in tables by col
    for t in range(1,T): # t -> col (time)

        for s in range(0,n): # s -> row (state)
            prob_event = math.log( em[s][o_index[obs[t]]] ) # log( bj(Ot+1))

            index, lmax = argmax(ls[:, t-1], tm[s][:])

            value = prob_event + lmax
            ls[s][t] = value
            phi[s][t] = index

    last_col = ls[:, np.shape(ls)[1]-1]

    start = np.argmax(last_col)
    path = [start]

    for time in range(T-1, 0, -1):
        start = phi[int(start)][time]
        path = np.insert(path, 0, int(start))
    
    return path

In [316]:
# set inital states
init = np.zeros(len(s_index))
init[s_index['DET']] = 1

In [317]:
path = viterbi(em, tm, flat_corp, init)

In [318]:
# map index -> POS
def decode(path):
    tags = []
    for i in path:
        tags.append(s_index_n[i])
    
    return tags

In [319]:
# get correct POS from NLTK
valid = []
for w in flat_brown:
    valid.append(w[1])

In [320]:
test = decode(path)

In [321]:
# compute the number of correct tags against NLTK tagger
correct = 0
total = 0
for i in range(len(test)):
    if test[i] == valid[i]:
        correct += 1
    total += 1
    
c = correct * 1.0/total * 100
print("tagger accuracy: " + str(c) + "%")

tagger accuracy: 33.2627245062%
